In [2]:
import requests

def fetch_data(
        buys_from="",
        buys_to="",
        freeze_authority=False,
        lp_burned_perc=True,
        mint_authority=True,
        mkt_cap_from=0,
        mkt_cap_to=float("inf"),
        social=True,
        order_by="created_at",
        order_dir="desc",
        sells_from=0,
        sells_to=float("inf"),
        pump=False,
        top_holders_perc=0,
        txns_from=0,
        txns_to=float("inf"),
        usd_liq_from=0,
        usd_liq_to=float("inf"),
        volume_from=0,
        volume_to=float("inf"),
        cookie="",
        user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36",
        accept="application/json, text/plain, */*",
        referer="https://photon-sol.tinyastro.io/",
        url="https://photon-sol.tinyastro.io/api/discover/search"
    ):
    """
    Fetch data from the given URL with the specified parameters.
    """

    headers = {
        "Cookie": cookie,
        "User-Agent": user_agent,
        "Accept": accept,
        "Referer": referer,
        "Connection": "keep-alive",
        "Accept-Language": "en-US,en;q=0.9"
    }

    params = {
        "buys_from": buys_from,
        "buys_to": buys_to,
        "freeze_authority": freeze_authority,
        "lp_burned_perc": lp_burned_perc,
        "mint_authority": mint_authority,
        "mkt_cap_from": mkt_cap_from,
        "mkt_cap_to": mkt_cap_to,
        "social": social,
        "order_by": order_by,
        "order_dir": order_dir,
        "sells_from": sells_from,
        "sells_to": sells_to,
        "pump": pump,
        "top_holders_perc": top_holders_perc,
        "txns_from": txns_from,
        "txns_to": txns_to,
        "usd_liq_from": usd_liq_from,
        "usd_liq_to": usd_liq_to,
        "volume_from": volume_from,
        "volume_to": volume_to
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

In [3]:
import pandas as pd
from pandas import json_normalize

def flatten_data(data):
    """
    Flatten the nested data into a pandas DataFrame using json_normalize.

    Args:
        data (dict): The data object containing nested dictionaries and lists.

    Returns:
        pd.DataFrame: A fully flattened DataFrame.
    """
    df_flat = json_normalize(data["data"])
    return df_flat

In [18]:
def filter_data(df_flat):
    """
    filter out rows where attributes.socials.twitter is NaN or None.

    Args:
        data (dataframe): The dataframe containing the coin info

    Returns:
        pd.DataFrame: A filtered and flattened DataFrame.
    """
    
    # Filter out rows where 'attributes.socials.twitter' is NaN or None
    df_filtered = df_flat[df_flat['attributes.socials.twitter'].notna() & df_flat['attributes.socials.twitter'].apply(lambda x: x is not None)]
    df_filtered = df_filtered[df_filtered['attributes.socials.website'].notna() & df_filtered['attributes.socials.website'].apply(lambda x: x is not None)]
    df_filtered = df_filtered[df_filtered['attributes.cur_liq.usd'] >= 5000]
    df_filtered = df_filtered.reset_index(drop=True)
    return df_filtered


In [19]:
# Example usage:
if __name__ == "__main__":
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.expand_frame_repr', False)
    data = fetch_data(
        cookie="_photon_eth_production=LZAZyE9MO1JH3jJy%2BRiMF89XRvl8knj3XA9mDN%2FkzNJR4F4mZiazjVcvnCTiWVVPWDslLV0hoEFoJm2kbUkh6WJjU8HV5dkV7R87DDR%2F8XtiKq0Lp%2FMNOUnKhEaQkT28rWUOGJHAN8yuOYCH4iJuxTJBRvL%2FNjx5JFaB%2FfxqlZnDNwUfVouYFp%2FO8wiJKwpWbQcYSeJ2m9FLQTraykbm5hfOaXWb4aza2dV6ekZ8X55hRiijpAZgA5My4q5Us8oS95PWO%2Br0LsGNV%2F2wccU3BQ%2F5QLmOWuLwTD6oRqN5w3QyK80gdbRmvIfO%2BW0A0O8yfx%2Ft3ntA3q8IvxI8SWEFvDnaCcAl5IQj--TQItHumozBYy%2F7pI--cQbaq0lbeaQUTKE8dj9nGg%3D%3D; trid=1f7eb184a0403031f0412a4cefc81816; wallet.address=eyJfcmFpbHMiOnsibWVzc2FnZSI6IklrWkZObVZUWlZKS1NEWXpNM2gwTVZkS1UwTnpOVkJDTm1FME9XWTRUVXBHVm05dFEyUlRjR1YxVkdObUlnPT0iLCJleHAiOiIyMDI0LTA1LTI0VDE2OjQ0OjEyLjAwMFoiLCJwdXIiOiJjb29raWUud2FsbGV0LmFkZHJlc3MifX0%3D--81dc48ff7f3b4d19e9ca868ac72dba24c6a68590; __cf_bm=6diTSupuRIPmRCIsBFZEWWJjHSVOZJIpmbEXwPxTiqU-1714839454-1.0.1.1-9ZxwFpYxRU4JtwQG452RcMMgVJ.eIBJUxCn3LThVpAvxOrG.S4P3_v4ukdLb5A20qseH5mGZgZnXfJ0jhVf9BQ; cf_clearance=8dDRUigdHt2O2nqDi6707_UENAk67wsV62L1iI7eGXw-1714839455-1.0.1.1-6_XAOB9xNj2NhsvZRTJ6VPkFiAkrGgcFkQjOgwVDGSXx7g.MwCh3PozOOsHNV.GzRJPAtxxnXfjGNkCaVBgR1w; _photon_ta=XjzBwbZO6Ht8%2BhcC3%2BNtxcykHs2xaC432gMlEspb0rSQLu1gTj%2F7VfMxEYkbo6BQvpTuH222EMqr%2BE97azmA16TE%2FzYFSQhUkbuh%2F1OB7EfIpOxAJ4A4Q47k%2FpFWiMY8rI0Xvpcn8iQj5eDE2hikF0OXZMNFSn06X4u8viLZeY5cr0EpXIpPOrrbczBYxf6DKlRmTJqyuVIVTyPJPpqxbLsL1VnuAdvN88%2Bzl61WAtVk2RxwFFOisqEPifI0%2ByTRF%2BhiOBE9mTnGlEQLs6IlDep0fMkCyE%2BhouSReEaRdEHy9q2HysDLTeNDbBB8K%2BtfRmODpnwzn5aXO%2FK%2B%2F3uR1Gtm1XSuQkgDewpHdkuivps6eWUJAR1azqZdAnGgQ%2BBCGSw%2BUQgi2a2OnLgjpVRMPiAkzi%2FC6h%2Fi%2BW%2BcTikvISmMHSc%2Fsy8hDFaSTCkvN5RThap9NZti0iEpg3%2FVRkju6SEYMgEyi6MRDftZRenG%2BFp8cjVu1jGX3P6NA7g6AXfFS5I%2BKEQX1Bjmatr1VZH%2BIxtUEjid610mTG6A02s30nE%3D--Y%2FhYs0z%2FoGL1JLgK--493920NiSnv%2Fpv3fdWIEVQ%3D%3D",
    )
    flattened_df = flatten_data(data)
    filtered_df = filter_data(flattened_df)
    display(filtered_df)

,id,type,attributes.volume,attributes.buys_count,attributes.sells_count,attributes.address,attributes.tokenAddress,attributes.fdv,attributes.price_usd,attributes.name,attributes.symbol,attributes.created_timestamp,attributes.open_timestamp,attributes.init_liq.usd,attributes.init_liq.quote,attributes.init_liq.token,attributes.init_liq.lp_amount,attributes.init_liq.timestamp,attributes.init_liq.percentage,attributes.init_liq.open_timestamp,attributes.cur_liq.quote,attributes.cur_liq.usd,attributes.audit.mint_authority,attributes.audit.freeze_authority,attributes.audit.lp_burned_perc,attributes.audit.top_holders_perc,attributes.socials.twitter,attributes.socials.website,attributes.socials.telegram,attributes.socials.medium,attributes.socials.reddit,attributes.dex_i,attributes.ignored,attributes.fromPump,attributes.socials
0,406459,,9.674685e+04,508,376,7ZWUPXFQD4E8mDLgmaaHDTmuZUj1ns86donYjVhLHCdP,5nZBpGpM4QyCCwxkKxBwyJZ24ohn2q9qfaKQbJ6ncS3p,63358.873605,0.063359,Book Of JPOW,BOJP,1714843429,1714843427,1463.5,10.0,1000000.0,70709678118,1714843429,1.0,1714843427,95.421589,13936.323144,False,False,100,16.8427,https://twitter.com/BookOfJPOW,https://www.bookofjpow.xyz/,https://t.me/BookOfJPOW,NaN,NaN,0,False,False,NaN
1,406256,,2.926913e+05,979,693,6aE4Ta9ikQVKNj2aRubkXVPmm4qQwMw9Ji7o9eLfypHu,7rgh653etwFRjvFSLGNFc8UNXuRdJjkEnJJwdVDPd6sD,32779.703760,0.000037,richi,richi,1714842399,0,23164.3713285816,158.010718476,206900000.0,4042044499673,1714842399,0.2155208333333333,0,132.665995,19375.868631,False,False,100,30.7044,https://twitter.com/richiRaccoon,https://richisolana.xyz/,https://t.me/richionsolana,NaN,NaN,0,False,True,NaN
2,406084,,6.421362e+05,1630,1227,EpYAxZuF6UgZnWX74fu93r8Sawx4nuxQd7xcGzvoYqrD,GZ7SZmDVZpWDTA7dWHUPQQcPSvMK4d9W6Hpvbq3ytFqS,103933.297271,0.000106,Niggas in Paris,NIP,1714841445,0,23175.43207682154,158.010718462,206900000.0,4042044499494,1714841445,0.2069,0,228.148700,33321.117700,False,False,100,35.9760,https://twitter.com/Sol_NIP,https://www.youtube.com/watch?v=gG_dA32oH44,None,NaN,NaN,0,False,True,NaN
3,406068,,1.246438e+05,757,564,FwzNa9AfAUwfEUZSuDAA76pi8QsgjKC1QLc6VZznyguJ,7qK2WYxLtQc3by98NqZvzpqbFzbizrtvhZemy2i7Sg3r,20935.912356,0.000209,Lil Brett aka BMoney,LBrett,1714841384,1714841340,1466.7,10.0,98000000.0,22134943621178,1714841384,0.9800000294000009,1714841340,54.726101,7992.747036,False,False,100,21.6586,https://x.com/liIbrett,https://lilbrett.net/,https://t.me/lilbrett,NaN,NaN,0,False,False,NaN
4,405627,,1.954462e+05,469,386,Gprt6NCnTwUVPshFhuo8TT5E8Eqre1CFjsWmcQ8zAACQ,2Q1agc4LATahzMqoKpG58iKKqyr31FJTF61Cr4iyiuFk,3422.019139,0.000003,Lil X,LILX,1714839218,0,23025.32185814408,158.010718214,206900000.0,4042044496321,1714839218,0.2069,0,40.041330,5848.036206,False,False,100,10.8341,https://twitter.com/elonmusk/status/1786788935...,https://twitter.com/elonmusk/status/1786788935...,https://twitter.com/elonmusk/status/1786788935...,NaN,NaN,0,False,True,NaN
5,405327,,8.450819e+05,1893,1488,5zCBfkCct5wRCjELxy1iww7wxLDERnqXZZqt3XghSgvB,385RSNkkNjZQBjRFTYs1jqfkDVGgUAmnRULfc8wux6WM,5744.139114,0.000006,Dailylive,Dailylive,1714837671,0,23031.64236704064,158.010718764,206900000.0,4042044503358,1714837671,0.2069,0,53.923342,7875.504030,False,False,100,15.0006,https://twitter.com/Dailylivesol,https://t.me/Dailylivesol,https://t.me/Dailylivesol,NaN,NaN,0,False,True,NaN
6,404782,,1.785069e+05,1565,1383,6zYa4ks5kns8KerbeXLFqDSzED7o8DrsKBkderzuXJCr,5mVGN4uVq84WwDrZLdNvFALLmgtvK5QRWTiB5D5ppozm,70621.385290,0.000071,Mett Fure,METTFURE,1714834951,1714834934,1468.5,10.0,510381254.0,50515396051183,1714834951,0.5103812545103813,1714834934,76.628436,11191.583126,False,False,100,24.5229,https://x.com/mettfure_sol,https://mettfure.site,https://t.me/mettfureonsol,NaN,NaN,0,False,False,NaN
7,404647,,6.992891e+04,417,291,58sDaWFLsDs7bM6zyxLn4XZuQEAketHRY7rUhqo1apHp,GosK5CuUokmpH8vPkwFgC6pnwdNZ6HNxCVVhf6KooAoe,28667.753821,0.000029,Bulgaria Coin,BULGARIA,1714834222,0,23203.8741988119,158.010719774,206900000.0,4042044516279,1714834222